In [1]:
import numpy as np 
from implementations import *
from helpers import *
from helper_functions import *
from optimization import *
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
# load the training dataset
print("loading the training dataset")
DATA_TRAIN_PATH = 'train.csv'
y_tr, tx_tr, ids_tr = load_csv_data(DATA_TRAIN_PATH)
print("training dataset loaded")

loading the training dataset
training dataset loaded


In [4]:
# load the test dataset
print("loading the test dataset")
DATA_TEST_PATH = 'test.csv'
y_te, tx_te, ids_te = load_csv_data(DATA_TEST_PATH)
print("test dataset loaded")

loading the test dataset
test dataset loaded


In [7]:
# printing the shapes of the arrays
print(f'the shape of the training y is {y_tr.shape} and the training tx is {tx_tr.shape}')
print(f'the shape of the test y is {y_te.shape} and the test tx is {tx_te.shape}')

the shape of the training y is (250000,) and the training tx is (250000, 30)
the shape of the test y is (568238,) and the test tx is (568238, 30)


In [7]:
np.set_printoptions(formatter={'float': lambda x: "{0:10.3f}".format(x)})

def print_col_info(t):
    mean_ = np.nanmean(t, axis=0)
    max_ = np.nanmax(t, axis=0)
    min_ = np.nanmin(t, axis=0)
    median_ = np.nanmedian(t, axis=0)
    std_ = np.nanstd(t, axis=0)
    print(t.shape)

    table = np.vstack([range(t.shape[1]), mean_, max_, min_, median_, std_])

    for line in table.T:
        print(line)

In [8]:
# testing the mean square gd
print("test mean square GD") 
#y_tr[np.where(y_tr == 0)] = -1
y_tr[np.where(y_tr == -1)] = 0
txOpti = dataClean_without_splitting(tx_tr)

initial_w = np.zeros(txOpti.shape[1])
max_iters = 100
gamma = 0.3 #best gamma here
w, loss = mean_squared_error_gd(y_tr, txOpti, initial_w, max_iters, gamma)
print("end of the mean_squared_error_gd with w=",w," and loss=", loss)

label = predict(w, txOpti)
y_tr[np.where(y_tr == 0)] = -1
acc = calculate_accuracy(y_tr, label)
print("the accuracy of the method is acc=",acc)
print("end of mean square GD")

test mean square GD
Current iteration=0	 the loss=0.1176670	 the grad=0.5271830
Current iteration=10	 the loss=0.0819012	 the grad=0.0148576
Current iteration=20	 the loss=0.0817723	 the grad=0.0031852
Current iteration=30	 the loss=0.0817611	 the grad=0.0012227
Current iteration=40	 the loss=0.0817590	 the grad=0.0005862
Current iteration=50	 the loss=0.0817585	 the grad=0.0003265
Current iteration=60	 the loss=0.0817583	 the grad=0.0001973
Current iteration=62	 the loss=0.0817583	 the grad=0.0001794
end of the mean_squared_error_gd with w= [     0.343      0.090     -0.120      0.030     -0.026     -0.011
     -0.004     -0.016     -0.015     -0.013     -0.014     -0.040
      0.053      0.063      0.042     -0.001     -0.000     -0.005
      0.002      0.002     -0.022     -0.000      0.005      0.026
      0.020     -0.002     -0.001     -0.001     -0.000     -0.001
      0.014]  and loss= 0.0817582975816111
the accuracy of the method is acc= 0.761
end of mean square GD


In [15]:
#testing mean square GD with spliting the data 
print("test mean square GD with spliting") 
#y_tr[np.where(y_tr == 0)] = -1
y_tr[np.where(y_tr == -1)] = 0

txOpti, yOpti, idsOpti = dataClean(tx_tr, y_tr)
ws = []
losses = []
max_iters = 200
gamma = 0.3

for i in range(4): 
    print("shape of txOpti[i] ", txOpti[i].shape)
    initial_w = np.zeros(txOpti[i].shape[1])
    # initial_w = ws
    w, loss = mean_squared_error_gd(yOpti[i], txOpti[i], initial_w, max_iters, gamma)
    losses.append(loss) 
    ws.append(w)
    print("end of the least_square with w=",w," and loss=", loss)
    
accs = []
labels = []
for i in range(4): 
    label = predict(ws[i], txOpti[i])
    yOpti[i][np.where(yOpti[i] == 0)] = -1
    acc = calculate_accuracy(yOpti[i], label)
    print("the accuracy on the train set is ", acc)
    accs.append(acc)
    labels.append(label)
    
accTot = (accs[0] + accs[1] + accs[2] + accs[3])/4
print("the total accuracy on the train set is ", accTot)

test mean square GD with spliting
shape of txOpti[i]  (99913, 19)
Current iteration=0	 the loss=0.0882093	 the grad=0.4316712
Current iteration=25	 the loss=0.0694140	 the grad=0.0023148
Current iteration=50	 the loss=0.0694055	 the grad=0.0003233
Current iteration=58	 the loss=0.0694053	 the grad=0.0001792
end of the least_square with w= [     0.255      0.030     -0.062     -0.000     -0.005      0.032
     -0.005      0.078     -0.076     -0.001      0.018     -0.001
     -0.002     -0.037      0.008     -0.001     -0.087     -0.002
      0.016]  and loss= 0.06940534792972657
shape of txOpti[i]  (77544, 23)
Current iteration=0	 the loss=0.1272059	 the grad=0.4789405
Current iteration=25	 the loss=0.0908357	 the grad=0.0031104
Current iteration=50	 the loss=0.0908197	 the grad=0.0004366
Current iteration=62	 the loss=0.0908195	 the grad=0.0001722
end of the least_square with w= [     0.357      0.119     -0.108      0.006     -0.004     -0.027
      0.011     -0.013     -0.046      0

In [16]:
#test mean square SGD
print("test mean square SGD")
y_tr[np.where(y_tr == 0)] = -1
#y_tr[np.where(y_tr == -1)] = 0
txOpti = dataClean_without_splitting(tx_tr)

initial_w = np.zeros(txOpti.shape[1])
max_iters = 200
gamma = 0.03
w, loss = mean_squared_error_sgd(y_tr, txOpti, initial_w, max_iters, gamma)
print("end of the mean_squared_error_sgd with w=",w," and loss=", loss)

label = predict(w, txOpti)
#y_tr[np.where(y_tr == 0)] = -1
acc = calculate_accuracy(y_tr, label)
print("the accuracy of the method is acc=",acc)
print("end of mean square SGD")

test mean square SGD
Current iteration=0	 the loss=0.0848699	 the grad=4.4272075
Current iteration=25	 the loss=0.0113748	 the grad=1.62929178
Current iteration=50	 the loss=0.0022188	 the grad=1.5002008
Current iteration=75	 the loss=0.0637552	 the grad=8.31897734
Current iteration=100	 the loss=0.0557767	 the grad=4.2659434
Current iteration=125	 the loss=0.0631818	 the grad=6.8008771
Current iteration=150	 the loss=0.1368293	 the grad=6.86031757
Current iteration=175	 the loss=0.0101451	 the grad=1.78398114
end of the mean_squared_error_sgd with w= [    -0.145      0.137     -0.256      0.020     -0.048     -0.137
      0.324     -0.614      0.063     -0.062     -0.074      0.493
      0.147      0.262     -0.049      0.064     -0.064      0.033
     -0.211      0.159     -0.086      0.182      0.041      0.129
      0.119     -0.009     -0.210      0.031      0.364      0.114
      0.056]  and loss= 0.02758508732088929
the accuracy of the method is acc= 0.649984
end of mean square 

In [17]:
#testing mean square SGD with spliting the data 
print("test mean square SGD with spliting") 
y_tr[np.where(y_tr == 0)] = -1
#y_tr[np.where(y_tr == -1)] = 0

txOpti, yOpti, idsOpti = dataClean(tx_tr, y_tr)
ws = []
losses = []
max_iters = 100
gamma = 0.03

for i in range(4): 
    print("shape of txOpti[i] ", txOpti[i].shape)
    initial_w = np.zeros(txOpti[i].shape[1])
    w, loss = mean_squared_error_sgd(yOpti[i], txOpti[i], initial_w, max_iters, gamma)
    losses.append(loss) 
    ws.append(w)
    print("end of the least_square with w=",w," and loss=", loss)
    
accs = []
labels = []
for i in range(4): 
    label = predict(ws[i], txOpti[i])
    #yOpti[i][np.where(yOpti[i] == 0)] = -1
    acc = calculate_accuracy(yOpti[i], label)
    print("the accuracy on the train set is ", acc)
    accs.append(acc)
    labels.append(label)
    
accTot = (accs[0] + accs[1] + accs[2] + accs[3])/4
print("the total accuracy on the train set is ", accTot)

test mean square SGD with spliting
shape of txOpti[i]  (99913, 19)
Current iteration=0	 the loss=0.0412636	 the grad=4.8741653
Current iteration=25	 the loss=0.1833951	 the grad=5.90437427
Current iteration=50	 the loss=0.1698078	 the grad=4.3910432
Current iteration=75	 the loss=0.0244233	 the grad=0.6722776
end of the least_square with w= [    -0.461     -0.001     -0.121     -0.007     -0.089     -0.150
     -0.089      0.123     -0.012     -0.066     -0.021     -0.125
     -0.117     -0.099     -0.014      0.135     -0.152     -0.043
      0.034]  and loss= 0.044599909673553054
shape of txOpti[i]  (77544, 23)
Current iteration=0	 the loss=0.0715228	 the grad=4.5526050
Current iteration=25	 the loss=0.0686201	 the grad=2.1498725
Current iteration=50	 the loss=0.0909811	 the grad=5.3024104
Current iteration=75	 the loss=0.1468967	 the grad=6.92107945
end of the least_square with w= [    -0.280      0.099     -0.378      0.060     -0.078     -0.109
     -0.011     -0.233     -0.089   

In [18]:
# test least square
print("test least square")
#y_tr[np.where(y_tr == 0)] = -1
y_tr[np.where(y_tr == -1)] = 0
txOpti = dataClean_without_splitting(tx_tr)
w, loss = least_squares(y_tr, txOpti)

label = predict(w, txOpti)
y_tr[np.where(y_tr == 0)] = -1
acc = calculate_accuracy(y_tr, label)
print("end of the least_squares with w=",w," and loss=", loss)
print("the accuracy of the method is acc=",acc)

test least square
end of the least_squares with w= [     0.343      0.091     -0.120      0.030     -0.026     -0.011
     -0.004     -0.016     -0.015     -0.013     -0.015     -0.040
      0.053      0.063      0.042     -0.001     -0.000     -0.005
      0.002      0.002     -0.022     -0.000      0.005      0.026
      0.020     -0.002     -0.001     -0.001     -0.000     -0.001
      0.014]  and loss= 0.08175819768998804
the accuracy of the method is acc= 0.760976


In [4]:
# test least square with spliting data
print("test the least square with spliting the dataset")
y_tr[np.where(y_tr == -1)] = 0
#y_tr[np.where(y_tr == 0)] = -1
txOpti, yOpti, idsOpti = dataClean(tx_tr, y_tr)
ws = []
losses = []

for i in range(4): 
    print("shape of txOpti[i] ", txOpti[i].shape)
    w, loss = least_squares(yOpti[i], txOpti[i])
    losses.append(loss) 
    ws.append(w)
    print("end of the least_square with w=",w," and loss=", loss)
    
accs = []
labels = []
for i in range(4): 
    label = predict(ws[i], txOpti[i])
    yOpti[i][np.where(yOpti[i] == 0)] = -1
    acc = calculate_accuracy(yOpti[i], label)
    print("the accuracy on the train set is ", acc)
    accs.append(acc)
    labels.append(label)
    
accTot = (accs[0] + accs[1] + accs[2] + accs[3])/4
print("the total accuracy on the train set is ", accTot)

test the least square with spliting the dataset
shape of txOpti[i]  (99913, 19)
end of the least_square with w= [ 2.55141974e-01  3.04645019e-02 -6.19698357e-02 -2.03505638e-04
  3.13853233e+02  3.24178718e-02 -3.13863690e+02  7.82816538e-02
 -7.56286022e-02 -6.91809911e-04  1.78507108e-02 -8.02198236e-04
 -1.91742772e-03 -3.72980905e-02  7.81616762e-03 -7.75765026e-04
 -8.73852713e-02 -2.33561476e-03  1.61625193e-02]  and loss= 0.06940501382214669
shape of txOpti[i]  (77544, 23)
end of the least_square with w= [ 3.57345507e-01  1.19906474e-01 -1.07631097e-01  5.59723769e-03
 -3.79662365e-03 -2.69521777e-02  1.14745969e-02 -1.35846280e-02
 -4.56191405e-02  4.95796856e-02  4.10161712e-02 -5.13594962e-04
  2.50543402e-04  1.12195754e-02 -2.61774255e-03  4.65891254e-03
 -1.19712234e-02  1.62412443e-03  2.33449007e-03 -2.01364464e+02
 -2.27352165e-03  1.18861529e-03  2.01402301e+02]  and loss= 0.09081924257946217
shape of txOpti[i]  (50379, 30)
end of the least_square with w= [ 5.10808075e

In [5]:
for i in range(4):
    print(ws[i].shape)
    print(txOpti[i].shape)

(19,)
(99913, 19)
(23,)
(77544, 23)
(30,)
(50379, 30)
(30,)
(22164, 30)


In [21]:
# test ridge regression 
print("test ridge regression")
y_tr[np.where(y_tr == -1)] = 0
txOpti = dataClean_without_splitting(tx_tr)
print("shape of txOpti ", txOpti.shape)

lambda_, degree = best_lambda_degree(y_tr, txOpti, 4, np.logspace(-6, 0, 30), np.arange(2,5), 1)
print("best lambda is ",lambda_," and the best degree is ", degree)

txOpti_poly = build_poly(txOpti, degree)

w, loss = ridge_regression(y_tr, txOpti_poly, lambda_)

label = predict(w, txOpti_poly)
y_tr[np.where(y_tr == 0)] = -1
acc = calculate_accuracy(y_tr, label)

print("end of the logistic_regression with w=",w," and loss=", loss)
print("the accuracy on the train set is ", acc)

#y_pred = predict(w, tx_te)
#y_pred[y_pred==0] = -1

#OUTPUT_PATH = 'sample-submission'
#create_csv_submission(ids_te, y_pred, OUTPUT_PATH)

test ridge regression
shape of txOpti  (250000, 31)
best lambda is  1e-06  and the best degree is  2
end of the logistic_regression with w= [     0.110      0.110      0.103     -0.109      0.020     -0.026
     -0.009     -0.032     -0.012     -0.016     -0.012     -0.054
     -0.054      0.040      0.063      0.048     -0.001     -0.000
      0.015      0.002      0.002     -0.031     -0.000      0.020
     -0.007      0.021     -0.002     -0.001      0.020     -0.000
     -0.000      0.081      0.110     -0.024     -0.000     -0.021
      0.008      0.004      0.007     -0.000     -0.005      0.003
      0.018      0.020      0.046     -0.004     -0.010     -0.004
     -0.001     -0.013     -0.008     -0.002      0.008      0.001
     -0.013      0.035     -0.004      0.009      0.000     -0.005
      0.002     -0.004     -0.030]  and loss= 0.07827237263520095
the accuracy on the train set is  0.78142


In [6]:
# test ridge regression with spliting the data
print("test ridge regression by spliting data")

y_tr[np.where(y_tr == -1)] = 0
txOpti, yOpti, idsOpti = dataClean(tx_tr, y_tr)

losses = []
ws = []
txPoly = [] 
for i in range(4): 
    print(f'shape of txOpti[{i}] {txOpti[i].shape}')
    lambda_, degree = best_lambda_degree(yOpti[i], txOpti[i], 4, np.logspace(-6, 0, 30), np.arange(1, 5), 1)
    print("best lambda is ",lambda_," and best degree is ", degree)
    t = build_poly(txOpti[i], degree)
    w, loss = ridge_regression(yOpti[i], t, lambda_)
    losses.append(loss) 
    ws.append(w) 
    txPoly.append(t)
    print(f'end of the ridge_regression with loss={loss}')


accs = []
labels = []
for i in range(4): 
    label = predict(ws[i], txPoly[i])
    yOpti[i][np.where(yOpti[i] == 0)] = -1
    acc = calculate_accuracy(yOpti[i], label)
    print("the accuracy on the train set is ", acc)
    accs.append(acc)
    labels.append(label)
    
accTot = (accs[0] + accs[1] + accs[2] + accs[3])/4
print("the total accuracy on the train set is ", accTot)

test ridge regression by spliting data
shape of txOpti[0] (99913, 19)
best lambda is  1e-06  and best degree is  1
end of the ridge_regression with loss=0.06940528855719576
shape of txOpti[1] (77544, 23)
best lambda is  1e-06  and best degree is  1
end of the ridge_regression with loss=0.0908194056759205
shape of txOpti[2] (50379, 30)
best lambda is  1e-06  and best degree is  1
end of the ridge_regression with loss=0.08717399062025641
shape of txOpti[3] (22164, 30)
best lambda is  1e-06  and best degree is  1
end of the ridge_regression with loss=0.08878703991906979
the accuracy on the train set is  0.8131574469788716
the accuracy on the train set is  0.7320875889817394
the accuracy on the train set is  0.747513845054487
the accuracy on the train set is  0.7285237321783071
the total accuracy on the train set is  0.7553206532983512


In [26]:
# test logistic regression

y_tr[np.where(y_tr == -1)] = 0
txOpti, yOpti, idsOpti = dataClean(tx_tr, y_tr)

max_iters = 100
gamma = 0.5
ws = []
losses = []

for i in range(4): 
    print("shape of txOpti[i] ", txOpti[i].shape)
    initial_w = np.zeros(txOpti[i].shape[1])
    w, loss = logistic_regression(yOpti[i], txOpti[i], initial_w, max_iters, gamma)
    ws.append(w)
    losses.append(loss)
    print("end of the logistic_regression with w=",w," and loss=", loss)

    
labels = []
accs = []
for i in range(4): 
    label = predict_logistic(ws[i], txOpti[i])
    yOpti[i][np.where(yOpti[i] == 0)] = -1
    acc = calculate_accuracy(yOpti[i], label)
    print("the accuracy on the train set is ", acc)
    accs.append(acc)
    labels.append(label)

accTot = (accs[0] + accs[1] + accs[2] + accs[3])/4
print("the total accuracy on the train set is ", accTot)

#y_pred = predict(w, teOpti)

#OUTPUT_PATH = 'sample-submission LR 2'
#create_csv_submission(ids_te, y_pred, OUTPUT_PATH)

shape of txOpti[i]  (99913, 19)
Preiteration, the loss=0.9023999996647266, the grad=0.42567363610337283
Current iteration=0	 the loss=0.8669964	 the grad=0.4256736
Current iteration=25	 the loss=0.8223110	 the grad=0.0370658
Current iteration=50	 the loss=0.8286025	 the grad=0.0130025
Current iteration=75	 the loss=0.8307620	 the grad=0.0060880
end of the logistic_regression with w= [    -1.523      0.096     -0.366      0.101     -0.025      0.292
     -0.025      0.444     -0.530      0.001      0.115     -0.008
     -0.014     -0.204      0.056     -0.002     -0.614     -0.019
      0.129]  and loss= 0.8315736795015117
shape of txOpti[i]  (77544, 23)
Preiteration, the loss=0.8736879810552399, the grad=0.3493400704524976
Current iteration=0	 the loss=0.8487077	 the grad=0.3493401
Current iteration=25	 the loss=0.8288509	 the grad=0.0252360
Current iteration=50	 the loss=0.8324185	 the grad=0.0087206
Current iteration=75	 the loss=0.8328594	 the grad=0.0042534
end of the logistic_regr

In [12]:
OUTPUT_PATH = 'logreg_nosplit_072'
create_csv_submission(ids_te, y_pred, OUTPUT_PATH)

In [34]:
# test regularized logistic regression
txOpti = dataClean_without_splitting(tx_tr)
print("shape of txOpti ", txOpti.shape)
print("shape of y ", y_tr.shape)
initial_w = np.zeros(txOpti.shape[1])
max_iters = 100
gamma = 0.5
lambda_ = 0.5

w, loss = reg_logistic_regression(y_tr, txOpti, lambda_, initial_w, max_iters, gamma)

label = predict_logistic(w, txOpti)
acc = calculate_accuracy(y_tr, label)

# print("end of the logistic_regression with w=",w," and loss=", loss)
print("end of the logistic_regression with loss=", loss)
print("the accuracy on the train set is ", acc)

teOpti = dataClean_without_splitting(tx_te)
print("shape of teOpti ", teOpti.shape)

y_pred = predict_logistic(w, teOpti)

OUTPUT_PATH = 'sample-submission LR regression r=1.5, gamma,lambda=0.5'
create_csv_submission(ids_te, y_pred, OUTPUT_PATH)

shape of txOpti  (250000, 31)
shape of y  (250000,)
Current iteration=0	 the loss=0.8374429	 the grad=0.4304113
Current iteration=25	 the loss=0.8338336	 the grad=0.0000000
Current iteration=50	 the loss=0.8338336	 the grad=0.0000000
Current iteration=75	 the loss=0.8338336	 the grad=0.0000000
end of the logistic_regression with loss= 0.8338335535064907
the accuracy on the train set is  0.22644
shape of teOpti  (568238, 31)


In [31]:
#test logistic regularized regression with spliting data
y_tr[np.where(y_tr == -1)] = 0
txOpti, yOpti, idsOpti = dataClean(tx_tr, y_tr)

max_iters = 100
gamma = 0.5
lambda_ = 1e-06
ws = []
losses = []

for i in range(4): 
    print(f"shape of txOpti[{i}] ", txOpti[i].shape)
    initial_w = np.zeros(txOpti[i].shape[1])
    w, loss = reg_logistic_regression(yOpti[i], txOpti[i], lambda_, initial_w, max_iters, gamma)
    ws.append(w)
    losses.append(loss)
    print("end of the reg_logistic_regression with w=",w," and loss=", loss)

    
labels = []
accs = []
for i in range(4): 
    label = predict_logistic(ws[i], txOpti[i])
    yOpti[i][np.where(yOpti[i] == 0)] = -1
    acc = calculate_accuracy(yOpti[i], label)
    print("the accuracy on the train set is ", acc)
    accs.append(acc)
    labels.append(label)

accTot = (accs[0] + accs[1] + accs[2] + accs[3])/4
print("the total accuracy on the train set is ", accTot)


shape of txOpti[i]  (99913, 19)
Current iteration=0	 the loss=0.8669964	 the grad=0.4256736
Current iteration=25	 the loss=0.8223108	 the grad=0.0370645
Current iteration=50	 the loss=0.8286017	 the grad=0.0130014
Current iteration=75	 the loss=0.8307608	 the grad=0.0060873
end of the reg_logistic_regression with w= [    -1.523      0.096     -0.366      0.101     -0.025      0.292
     -0.025      0.444     -0.530      0.001      0.115     -0.008
     -0.014     -0.204      0.056     -0.002     -0.614     -0.019
      0.129]  and loss= 0.8315723453390188
shape of txOpti[i]  (77544, 23)
Current iteration=0	 the loss=0.8487077	 the grad=0.3493401
Current iteration=25	 the loss=0.8288506	 the grad=0.0252352
Current iteration=50	 the loss=0.8324179	 the grad=0.0087200
Current iteration=75	 the loss=0.8328588	 the grad=0.0042530
end of the reg_logistic_regression with w= [    -0.792      0.646     -0.618      0.101     -0.014     -0.136
      0.068     -0.058     -0.241      0.361      0.1

In [33]:
print(np.array(ws).shape)

(4,)


/tmp/ipykernel_73/3621360310.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(ws).shape)
